In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy.io import arff
import pandas as pd
from scipy import stats
import itertools
import random
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import h5py
import os
import time
from sklearn.svm import *
import matplotlib.font_manager
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import Normalizer
from sklearn.metrics import roc_auc_score
import my_metrics

In [ ]:
# arr = ('/eos/cms/store/group/comm_dqm/cmsml4dc/2016-v1/data2016_v2/'+
#       'BTagCSV/crab_BTagCSVRun_2016C-18Apr2017-v1/170808_213346/0000/AODTree_102.npy')
# good_arr = '/mnt/hdf5test/BTagCSV_C_signal.h5'
# bad_arr  = '/mnt/hdf5test/BTagCSV_C_background.h5'

good_arr = '/mnt/hdf5test/JetHT_C_signal.h5'
bad_arr  = '/mnt/hdf5test/JetHT_C_background.h5'

# data = np.load(arr, encoding = 'latin1')
sig_jetfile = h5py.File(good_arr,'r')
sig_jet     = sig_jetfile['JetHT_C_signal'[:]]

bg_jetfile = h5py.File(bad_arr,'r')
bg_jet     = bg_jetfile['JetHT_C_background'[:]]

good_jets = np.empty([0,2802])
bad_jets  = np.empty([0,2802])
# print(bad_jets)
print("XXXXXX")
good_jets = np.concatenate((good_jets, sig_jet), axis=0)
bad_jets  = np.concatenate((bad_jets,  bg_jet),  axis=0)
# print(bad_jets)

df1 = pd.DataFrame(good_jets)
# cutted_df = df1.iloc[:, 0:100]
# df1 = cutted_df
df1['class'] = 0
print(df1)
df2 = pd.DataFrame(bad_jets)
# cutted_df = df2.iloc[:, 0:100]
# df2 = cutted_df
df2['class'] = 1

frames = [df1,df2]
df     = pd.concat(frames)
print(df)
# # arr = np.random.rand(10, 5)
# df = pd.DataFrame(data)
# print(df)

In [ ]:
# cutted_df = df.iloc[:, 0:100]
# df = cutted_df
df = shuffle(df)
print(df)

In [ ]:
df = df.reset_index(drop=True)
print(df)

In [ ]:
labels = df['class'].astype(int)
# labels[labels != 0] = 1
del df['class']

In [ ]:
print(df)

In [ ]:
print(labels[40:200])


In [ ]:
data = df 

In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(data)
data_n = pd.DataFrame(np_scaled)
data_n = data_n.astype('float32')

In [ ]:
print(data_n)

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

encoding_dim = 2000
input = Input(shape=(2802,))
encoded = Dense(encoding_dim, activation='relu')(input)
decoded = Dense(2802, activation='sigmoid')(encoded)
autoencoder = Model(inputs=input, outputs=decoded)

encoder = Model(inputs=input, outputs=encoded)

encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy', my_metrics.f1, my_metrics.mcor])


autoencoder.fit(data_n.values, data_n.values,
                epochs=2500,
                batch_size=1700,
                shuffle=True,
#                 verbose=0
               )



In [ ]:
autoencoder.save('my_model.h5')
autoencoder.save_weights('my_model_weights.h5')

In [ ]:
encoded = encoder.predict(data_n.values)
decoded = decoder.predict(encoded)


In [ ]:
print(encoded.shape)
print("XXXX")
print(decoded)
print("XXXX")
print()

In [ ]:
dist = np.zeros(len(data_n.values))
for i, x in enumerate(data_n.values):
    dist[i] = np.linalg.norm(x-decoded[i])

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(labels, dist)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10,6))
plt.plot(fpr, tpr, color='red', label='AUC = %0.2f)' % roc_auc)
plt.xlim((0,1))
plt.ylim((0,1))
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive rate')
plt.ylabel('True Positive rate')
plt.title('ROC Autoencoder 100-80-100 ReLU/Sigmoid synth\_multidim\_100\_000')
plt.legend(loc="lower right")
plt.show()

In [ ]:
data['labels'] = labels
data['dist'] = dist

plt.figure(figsize=(40,28))
plt.scatter(data.index, data['dist'], c=data['labels'], edgecolor='black', s=30)
plt.xlabel('Index')
plt.ylabel('Score')
plt.xlim((0,17730))
plt.title("Outlier Score")
plt.show()
plt.show()

In [ ]:
def compute_error_per_dim(point):
    p = np.array(data_n.iloc[point,:]).reshape(1,2802)
    encoded = encoder.predict(p)
    decoded = decoder.predict(encoded)
    return np.array(p - decoded)[0]

plt.figure(figsize=(12,7))
plt.plot(compute_error_per_dim(17000))
plt.xlim((0,2802))
plt.xlabel('Index')
plt.ylabel('Reconstruction error')
plt.title("Reconstruction error in each dimension of point 350")
plt.show()